In [1]:
"""Heatmap."""

import matplotlib.pyplot as plt
from scipy.sparse import load_npz
import pandas as pd

In [2]:
image_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-wsi-img.tiff'
cell_gene_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-gene.npz'
cell_info_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-info.csv'

In [3]:
cell_centroids = pd.read_csv(cell_info_path)
cell_centroids[['x', 'y']].values
coords = cell_centroids[['x', 'y']].values
color = load_npz(cell_gene_path).tocsc()[:, 0].toarray().flatten()